In [1]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [17]:
#Get all species
spc_url = ('https://data.cityofnewyork.us/resource/uvpi-gqnh.json?' +\
            '$select=distinct spc_common' +\
            '&$where=(health!=\'NaN\'' +\
            'and spc_common!=\'NaN\')').replace(' ', '%20')           
spc = pd.read_json(spc_url).iloc[:,0]


#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = "https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/pulse/bootstrap.min.css"

app = JupyterDash(__name__, external_stylesheets=[external_stylesheets])

app.layout = html.Div(children=[
    
    #header
    html.H1('DATA 608 - Module 4',style={'backgroundColor':'#636efa', 'color':'white'}),
    html.H4('Interactive Visualizations in Python'),


    
    #name & date
    html.Div(
        children=(
            html.P('Fan Xu'),
            html.P('3/28/2021'),
            html.Br()
        )
    ),    
    
    #Assignment Description
    html.H6('Build a dash app for a arborist studying the health of various tree species (as defined by the variable ‘spc_common’) across each borough (defined by the variable ‘borough’). This arborist would like to answer the following two questions for each species and in each borough:'),    
    html.Br(),
    
    #dropdown box for species
    html.P("Select a species:"),
    dcc.Dropdown(
        id='spc',
        value='shingle oak',
        options=[{'value': x, 'label': x} for x in spc], 
        style={'width': '50%'},
        clearable=False
    ), 
    html.Br(),

    
    
    #Div for Question 1
    html.Div([
        html.H6('1.What proportion of trees are in good, fair, or poor health according to the \'health\' variable?'),
        dcc.Graph(id='fig_1'),
    ]),
        
    #Div for Question 2
    html.Div([
        html.H6('2.Are stewards (steward activity measured by the \'steward\' variable) having an impact on the health of trees?'),
        dcc.Graph(id='fig_2'),
    ])
    
])

@app.callback(
    Output("fig_1", "figure"), 
    Output("fig_2", "figure"),
    Input("spc", "value"),
    #Input("spc_selection", "value")
)

def generate_plots(spc):
    
    #Fig 1
    health_url = ('https://data.cityofnewyork.us/resource/uvpi-gqnh.json?' +\
                  '$select=boroname, health, count(tree_id) as tree_cnt' +\
                  '&$where=(health!=\'NaN\'' +\
                  'and spc_common=\'' + spc + '\')' +\
                  '&$group=boroname, health').replace(' ', '%20')
    health = pd.read_json(health_url) 
    
    fig_1 = make_subplots(rows=1, cols=5, specs=[[{'type':'domain'}, 
                                            {'type':'domain'},
                                            {'type':'domain'},
                                            {'type':'domain'},
                                            {'type':'domain'}
                                           ]])
    fig_1.add_trace(go.Pie(labels=health.loc[health['boroname'] == 'Bronx']['health'], 
                         values=health.loc[health['boroname'] == 'Bronx']['tree_cnt'], name='Bronx', title='Bronx'),1, 1)
    fig_1.add_trace(go.Pie(labels=health.loc[health['boroname'] == 'Brooklyn']['health'],
                         values=health.loc[health['boroname'] == 'Brooklyn']['tree_cnt'], name='Brooklyn', title='Brooklyn'),1, 2)
    fig_1.add_trace(go.Pie(labels=health.loc[health['boroname'] == 'Manhattan']['health'],
                         values=health.loc[health['boroname'] == 'Manhattan']['tree_cnt'], name='Manhattan', title='Manhattan'),1, 3)
    fig_1.add_trace(go.Pie(labels=health.loc[health['boroname'] == 'Queens']['health'],
                         values=health.loc[health['boroname'] == 'Queens']['tree_cnt'], name='Queens', title='Queens'),1, 4)
    fig_1.add_trace(go.Pie(labels=health.loc[health['boroname'] == 'Staten Island']['health'],
                         values=health.loc[health['boroname'] == 'Staten Island']['tree_cnt'],
                         name='Staten Island', title='Staten Island'),1, 5)
    fig_1.update_traces(hole=0.4, hoverinfo="label+percent+name")
    fig_1.update_layout(title_text="Health Status of {} Across Boros in NY".format(spc))
    
    
    #Fig 2
    steward_url = ('https://data.cityofnewyork.us/resource/uvpi-gqnh.json?' +\
                  '$select=boroname, steward, health, count(tree_id) as tree_cnt' +\
                  '&$where=(health!=\'NaN\'' +\
                  'and spc_common=\'' + spc + '\')' +\
                  '&$group=boroname, steward, health').replace(' ', '%20')
    steward = pd.read_json(steward_url) 
    steward['tree_cnt(Log)'] = np.log(steward['tree_cnt'])
    
    fig_2=px.bar(steward,
                 x="steward",
                 y="tree_cnt(Log)",
                 color="health",
                 facet_row="health",
                 facet_col='boroname',
                 category_orders={"boroname": ["Bronx", "Brooklyn", "Manhattan", "Queesn","Staten Island"],
                                  "health": ["Good", "Fair","Poor"],
                                  "steward":['4orMore','3or4','1or2','None']},
                 hover_data=["tree_cnt"],
                 title='Steward Activity Count of {} Arocss Boros in NY (Log Transformed)'.format(spc)
                )
    fig_2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
                    
    return fig_1, fig_2

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True,port=8054)